In [1]:
from lale.datasets.openml import fetch

In [2]:
(X_train, y_train), (X_test, y_test) = fetch("jungle_chess_2pcs_raw_endgame_complete", "classification")

In [3]:
import ray
ray.shutdown()

In [4]:
# from lale.helpers import cross_val_score

In [5]:
# from lale.lib.sklearn import PCA, Nystroem, SelectKBest, RandomForestClassifier
# from lale.lib.lale import ConcatFeatures

# pipeline = (PCA() & Nystroem() & SelectKBest(k=3)) >> ConcatFeatures() >> RandomForestClassifier(n_estimators=200)
# # pipeline.visualize()

In [6]:
# sk_pipeline = pipeline.export_to_sklearn_pipeline()

In [7]:
# sk_pipeline

In [8]:
import ray
ray.init(object_store_memory=16 * 1024 * 1024 * 1024)

2021-05-25 15:55:50,487	INFO services.py:1269 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '9.163.5.112',
 'raylet_ip_address': '9.163.5.112',
 'redis_address': '9.163.5.112:6379',
 'object_store_address': '/tmp/ray/session_2021-05-25_15-55-48_881861_17264/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-05-25_15-55-48_881861_17264/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2021-05-25_15-55-48_881861_17264',
 'metrics_export_port': 63066,
 'node_id': 'd067798c6a3d62df2148c17631b1a63b58e53a141542f7697fe29297'}

In [9]:
import codeflare.pipelines.Datamodel as dm

In [10]:
from sklearn.model_selection import KFold, StratifiedKFold

In [11]:
kf = StratifiedKFold(n_splits=10)

In [12]:
pipeline = dm.Pipeline()

In [13]:
from sklearn.pipeline import FeatureUnion
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.kernel_approximation import Nystroem
from sklearn.feature_selection import SelectKBest

In [14]:
feature_union = FeatureUnion(transformer_list=[('PCA', PCA()),
                                                ('Nystroem',
                                                 Nystroem()),
                                                ('SelectKBest',
                                                 SelectKBest(k=3))])

In [15]:
random_forest = RandomForestClassifier(n_estimators=200)

In [16]:
node_fu = dm.EstimatorNode('feature_union', feature_union)
node_rf = dm.EstimatorNode('randomforest', random_forest)

In [17]:
pipeline.add_edge(node_fu, node_rf)

In [18]:
import codeflare.pipelines.Runtime as rt
from codeflare.pipelines.Runtime import ExecutionType

In [19]:
pipeline_input = dm.PipelineInput()
xy = dm.Xy(X_train, y_train)
pipeline_input.add_xy_arg(node_fu, xy)

In [20]:
scores = rt.cross_validate(kf, pipeline, pipeline_input)

In [21]:
scores

[0.8145188145188145,
 0.8115218115218116,
 0.8161838161838162,
 0.8168498168498168,
 0.8171828171828172,
 0.8175158175158175,
 0.8105228105228105,
 0.8211788211788211,
 0.8031312458361093,
 0.8154563624250499]

In [6]:
%%time 
cross_val_score(pipeline, X_train, y_train, cv=10)

CPU times: user 7min 45s, sys: 12.4 s, total: 7min 57s
Wall time: 7min 30s


[0.8168498168498168,
 0.8215118215118216,
 0.8138528138528138,
 0.8185148185148186,
 0.8178488178488178,
 0.8215118215118216,
 0.8121878121878122,
 0.8191808191808192,
 0.8021319120586275,
 0.8074616922051966]